# Final Exam

Recall the drawing system from lecture 18:

In [1]:
class Canvas:
    def __init__(self, width, height):
        self.width = width
        self.height = height
        self.data = [[' '] * width for i in range(height)]

    def set_pixel(self, row, col, char='*'):
        self.data[row][col] = char

    def get_pixel(self, row, col):
        return self.data[row][col]
    
    def h_line(self, x, y, w, **kargs):
        for i in range(x,x+w):
            self.set_pixel(i,y, **kargs)

    def v_line(self, x, y, h, **kargs):
        for i in range(y,y+h):
            self.set_pixel(x,i, **kargs)
            
    def line(self, x1, y1, x2, y2, **kargs):
        slope = 0
        if x2 != x1:
            slope = (y2-y1) / (x2-x1)
        if (slope > 0 or slope < 0) and y1 != y2 and x1 != x2:
            if x1 < x2:
                for x in range(x1,x2):
                    y= y1 + int((x - x1) * slope)
                    self.set_pixel(x,y, **kargs)
            elif x2 < x1:
                for x in range(x1,x2, -1):
                    y=y2 + int((x-x2) * slope)
                    self.set_pixel(x,y, **kargs)
        elif y2 == y1:
            if x1 > x2:
                self.h_line(x2, y2, x1-x2, **kargs)
            elif x2 > x1:
                self.h_line(x2, y2, x2-x1, **kargs)
        elif x2 == x1:
            if y1 > y2:
                self.v_line(x2, y2, y1-y2, **kargs)
            elif y2 > y2:
                self.v_line(x2, y2, y2-y1, **kargs)
                
                    
    def display(self):
        print("\n".join(["".join(row) for row in self.data]))

In [2]:
class Shape:
    def __init__(self, name="", **kwargs):
        self.name=name
        self.kwargs=kwargs
    
    def paint(self, canvas): pass

class Rectangle(Shape):
    def __init__(self, x, y, w, h, **kwargs):
        Shape.__init__(self, **kwargs)
        self.x = x
        self.y = y
        self.w = w
        self.h = h

    def paint(self, canvas):
        canvas.h_line(self.x, self.y, self.w, **self.kwargs)
        canvas.h_line(self.x, self.y + self.h, self.w, **self.kwargs)
        canvas.v_line(self.x, self.y, self.h, **self.kwargs)
        canvas.v_line(self.x + self.w, self.y, self.h, **self.kwargs)

class Square(Rectangle):
    def __init__(self, x, y, size, **kwargs):
        Rectangle.__init__(self, x, y, size, size, **kwargs)

class Line(Shape):
    def __init__(self, x1, y1, x2, y2,  **kwargs):
        Shape.__init__(self, **kwargs)
        self.x1=x1
        self.y1=y1
        self.x2=x2
        self.y2=y2
        
    def paint(self, canvas):
        canvas.line(self.x1,self.y1,self.x2,self.y2)
        
class CompoundShape(Shape):
    def __init__(self, shapes):
        self.shapes = shapes

    def paint(self, canvas):
        for s in self.shapes:
            s.paint(canvas)

In [3]:
class RasterDrawing:
    def __init__(self):
        self.shapes=dict()
        self.shape_names=list()
        
    def add_shape(self,shape):
        if shape.name == "":
            shape.name = self.assign_name()
        
        self.shapes[shape.name]=shape
        self.shape_names.append(shape.name)
        
    def paint(self,canvas):
        for shape_name in self.shape_names:
            self.shapes[shape_name].paint(canvas)
            
    def assign_name(self):
        name_base="shape"
        name = name_base+"_0"
        
        i=1
        while name in self.shapes:
            name = name_base+"_"+str(i)
            
        return name


1. Add `Point` and `Triangle` classes and test them.

In [4]:
class Point(Shape):
    def __init__(self, x, y, **kwargs):
        Shape.__init__(self, **kwargs)
        self.x = x
        self.y = y

    def paint(self, canvas):
        canvas.set_pixel(self.x, self.y, **self.kwargs)

class Triangle(Shape):
    def __init__(self, x1, y1, x2, y2, x3, y3, **kwargs):
        Shape.__init__(self, **kwargs)
        self.x1 = x1
        self.y1 = y1
        self.x2 = x2
        self.y2 = y2
        self.x3 = x3
        self.y3 = y3

    def paint(self, canvas):
        canvas.line(self.x1, self.y1, self.x2, self.y2, **self.kwargs)
        canvas.line(self.x2, self.y2, self.x3, self.y3, **self.kwargs)
        canvas.line(self.x3, self.y3, self.x1, self.y1, **self.kwargs)

In [5]:
c1=Canvas(50,25)
#s1=Square(5,5,20,char="^")
#s1.paint(c1)
p1=Point(20,20,char=".")
p1.paint(c1)
t1=Triangle(5,5,10,20,20,5, char=".")
t1.paint(c1)
t2=Triangle(2,5,8,25,2,35, char="+")
t2.paint(c1)
c1.display()

                                                  
                                                  
     ++++++++++++++++++++++++++++++               
        +                         +               
           +                    +                 
     .         +              +                   
     .  .         +          +                    
     .     .         +     +                      
     .        .          +                        
     .           .                                
     .              .                             
     .             .                              
     .           .                                
     .          .                                 
     .        .                                   
     .       .                                    
     .     .                                      
     .    .                                       
     .  .                                         
     . .                       

2. Add an `Arc` class that is instantiated with a center location, two axis lengths, and starting and ending angles. If start and end are not specified or are the same angle, the `Arc` instance should draw an oval. If in addition the two axes are the same, the `Arc` instance should draw a circle. Create `Oval` and `Circle` classes that inherit from `Arc`. Test everything.

In [6]:
import numpy as np
import math
class Arc(Shape):
    def __init__(self, x, y, rx, ry, start=None, stop=None, **kwargs):
        Shape.__init__(self, **kwargs)
        self.x = x
        self.y = y
        self.rx = rx
        self.ry = ry
        self.start = start
        self.stop = stop

    def paint(self, canvas):
        if self.start != None and self.stop != None and self.start != self.stop:
            dt = (self.stop - self.start)/360
            for theta in np.arange(self.start, self.stop, dt):
                x = int(self.rx * math.cos(theta) + self.x)
                y = int(self.ry * math.sin(theta) + self.y)
                canvas.set_pixel(x,y,**self.kwargs)
        else:
            dt = 2*math.pi/360
            for theta in np.arange(0, 2*math.pi, dt):
                x = int(self.rx * math.cos(theta) + self.x)
                y = int(self.ry * math.sin(theta) + self.y)
                canvas.set_pixel(x,y,**self.kwargs)
class Oval(Arc):
    def __init__(self, x, y, rx, ry, **kwargs):
        Arc.__init__(self, x, y, rx, ry, start=None, stop=None, **kwargs)
class Circle(Arc):
    def __init__(self, x, y, r, **kwargs):
        Arc.__init__(self, x, y, r, r, start=None, stop=None, **kwargs)

In [7]:
c1=Canvas(70,50)
# normal arc
# draws (part of) an oval but looks like a circle
a1=Arc(10,15,5,10,0,1.5*math.pi, char="1")
a1.paint(c1)
# arc w/ same start and end
a2=Arc(10,40,5,10,math.pi,math.pi, char="2")
a2.paint(c1)
# arc w/o start and end
# draws an oval but looks like a circle
a3=Arc(25,15,5,10,char="3")
a3.paint(c1)
o1=Oval(25,40,5,10,char="4")
o1.paint(c1)
# arc with same rx and ry
# draws a circle but looks like an oval
a4=Arc(40,15,5,5,char="5")
a4.paint(c1)
cr1=Circle(40,40,5,char="6")
cr1.paint(c1)
c1.display()

                                                                      
                                                                      
                                                                      
                                                                      
                                                                      
         111111111111             222222222222                        
       11            11         22            22                      
     11                11     22                22                    
     1                  1     2                  2                    
     1                  1     2                  2                    
                        11    2                  22                   
                        1     2                  2                    
                       11     22                22                    
                     11         22            22                      
      

3. Use your classes to create a `RasterDrawing` that draws a happy face.

In [8]:
c1=Canvas(31,31)
rd=RasterDrawing()

rd.add_shape(Circle(15,15,15,char="1",name="head"))
rd.add_shape(Oval(10,18,2,4,char="2",name="left eye"))
rd.add_shape(Oval(20,18,2,4,char="3",name="right eye"))
rd.add_shape(Arc(15,15,10,10,math.pi,2*math.pi,char="4",name="mouth"))

rd.paint(c1)

c1.display()

         111111111111          
       111          111        
      11              11       
    11                  11     
   11                    11    
   1      444444          1    
  1      44                1   
 11    44                  11  
 1     4      22222222      1  
11    4       2      2      11 
1    44       2      22      1 
1    4        22222222       1 
1    4            2          1 
1    4                       1 
1    4                       1 
1    4                       11
1    4                       1 
1    4                       1 
1    4        33333333       1 
1    44       3      3       1 
11    4       3      33     11 
 1     4      33333333      1  
 11    44         3        11  
  1      44                1   
   1      44444           1    
   11                    11    
    11                  11     
      11              11       
       111          111        
         111111111111          
               1               


4. Add to the `Shape` base class a `__str__()` method. Overwrite the method in each shape to generate a string of the python code necessary to reinstantiate the object. For example, for a rectangle originally instantiated using `Square(5,5,20,char="^")`, `__str__()` should return the string `'Square(5,5,20,char="^")'`.


In [9]:
class Shape:
    def __init__(self, name="", **kwargs):
        self.name=name
        self.kwargs=kwargs
    
    def paint(self, canvas): pass
    
    def __str__(self): pass

class Rectangle(Shape):
    def __init__(self, x, y, w, h, **kwargs):
        Shape.__init__(self, **kwargs)
        self.x = x
        self.y = y
        self.w = w
        self.h = h

    def paint(self, canvas):
        canvas.h_line(self.x, self.y, self.w, **self.kwargs)
        canvas.h_line(self.x, self.y + self.h, self.w, **self.kwargs)
        canvas.v_line(self.x, self.y, self.h, **self.kwargs)
        canvas.v_line(self.x + self.w, self.y, self.h, **self.kwargs)
    
    def __str__(self):
        string = 'Rectangle({},{},{},{},'.format(self.x,self.y,self.w,self.h)
        for key in self.kwargs.keys():
            if isinstance(self.kwargs[key],str):
                string += key + "='" + self.kwargs[key] + "',"
            else:
                string += key + '=' + self.kwargs[key] + ','
        if self.name != '':
            string += "name='" + self.name + "')"
        else:
            string = string[:-1] + ")"
        return string

class Square(Rectangle):
    def __init__(self, x, y, size, **kwargs):
        Rectangle.__init__(self, x, y, size, size, **kwargs)
    
    def __str__(self):
        string = 'Square({},{},{},'.format(self.x,self.y,self.w)
        for key in self.kwargs.keys():
            if isinstance(self.kwargs[key],str):
                string += key + "='" + self.kwargs[key] + "',"
            else:
                string += key + '=' + self.kwargs[key] + ','
        if self.name != '':
            string += "name='" + self.name + "')"
        else:
            string = string[:-1] + ")"
        return string

class Line(Shape):
    def __init__(self, x1, y1, x2, y2,  **kwargs):
        Shape.__init__(self, **kwargs)
        self.x1=x1
        self.y1=y1
        self.x2=x2
        self.y2=y2
        
    def paint(self, canvas):
        canvas.line(self.x1,self.y1,self.x2,self.y2)
    
    def __str__(self):
        string = 'Line({},{},{},{},'.format(self.x1,self.y1,self.x2,self.y2)
        for key in self.kwargs.keys():
            if isinstance(self.kwargs[key],str):
                string += key + "='" + self.kwargs[key] + "',"
            else:
                string += key + '=' + self.kwargs[key] + ','
        if self.name != '':
            string += "name='" + self.name + "')"
        else:
            string = string[:-1] + ")"
        return string
        
class CompoundShape(Shape):
    def __init__(self, shapes):
        self.shapes = shapes

    def paint(self, canvas):
        for s in self.shapes:
            s.paint(canvas)
    
    def __str__(self):
        string = 'CompoundShape(['
        for shape in self.shapes:
            string += str(shape) + ','
        string = string[:-1] + '])'
        return string

class Point(Shape):
    def __init__(self, x, y, **kwargs):
        Shape.__init__(self, **kwargs)
        self.x = x
        self.y = y

    def paint(self, canvas):
        canvas.set_pixel(self.x, self.y, **self.kwargs)
    
    def __str__(self):
        string = 'Point({},{},'.format(self.x,self.y)
        for key in self.kwargs.keys():
            if isinstance(self.kwargs[key],str):
                string += key + "='" + self.kwargs[key] + "',"
            else:
                string += key + '=' + self.kwargs[key] + ','
        if self.name != '':
            string += "name='" + self.name + "')"
        else:
            string = string[:-1] + ")"
        return string

class Triangle(Shape):
    def __init__(self, x1, y1, x2, y2, x3, y3, **kwargs):
        Shape.__init__(self, **kwargs)
        self.x1 = x1
        self.y1 = y1
        self.x2 = x2
        self.y2 = y2
        self.x3 = x3
        self.y3 = y3

    def paint(self, canvas):
        canvas.line(self.x1, self.y1, self.x2, self.y2, **self.kwargs)
        canvas.line(self.x2, self.y2, self.x3, self.y3, **self.kwargs)
        canvas.line(self.x3, self.y3, self.x1, self.y1, **self.kwargs)
    
    def __str__(self):
        string = 'Triangle({},{},{},{},{},{},'.format(self.x1,self.y1,self.x2,self.y2,self.x3,self.y3)
        for key in self.kwargs.keys():
            if isinstance(self.kwargs[key],str):
                string += key + "='" + self.kwargs[key] + "',"
            else:
                string += key + '=' + self.kwargs[key] + ','
        if self.name != '':
            string += "name='" + self.name + "')"
        else:
            string = string[:-1] + ")"
        return string

class Arc(Shape):
    def __init__(self, x, y, rx, ry, start=None, stop=None, **kwargs):
        Shape.__init__(self, **kwargs)
        self.x = x
        self.y = y
        self.rx = rx
        self.ry = ry
        self.start = start
        self.stop = stop

    def paint(self, canvas):
        if self.start != None and self.stop != None and self.start != self.stop:
            dt = (self.stop - self.start)/360
            for theta in np.arange(self.start, self.stop, dt):
                x = int(self.rx * math.cos(theta) + self.x)
                y = int(self.ry * math.sin(theta) + self.y)
                canvas.set_pixel(x,y,**self.kwargs)
        else:
            dt = 2*math.pi/360
            for theta in np.arange(0, 2*math.pi, dt):
                x = int(self.rx * math.cos(theta) + self.x)
                y = int(self.ry * math.sin(theta) + self.y)
                canvas.set_pixel(x,y,**self.kwargs)
    
    def __str__(self):
        string = 'Arc({},{},{},{},{},{},'.format(self.x,self.y,self.rx,self.ry,self.start,self.stop)
        for key in self.kwargs.keys():
            if isinstance(self.kwargs[key],str):
                string += key + "='" + self.kwargs[key] + "',"
            else:
                string += key + '=' + self.kwargs[key] + ','
        if self.name != '':
            string += "name='" + self.name + "')"
        else:
            string = string[:-1] + ")"
        return string

class Oval(Arc):
    def __init__(self, x, y, rx, ry, **kwargs):
        Arc.__init__(self, x, y, rx, ry, start=None, stop=None, **kwargs)
    
    def __str__(self):
        string = 'Oval({},{},{},{},'.format(self.x,self.y,self.rx,self.ry)
        for key in self.kwargs.keys():
            if isinstance(self.kwargs[key],str):
                string += key + "='" + self.kwargs[key] + "',"
            else:
                string += key + '=' + self.kwargs[key] + ','
        if self.name != '':
            string += "name='" + self.name + "')"
        else:
            string = string[:-1] + ")"
        return string

class Circle(Arc):
    def __init__(self, x, y, r, **kwargs):
        Arc.__init__(self, x, y, r, r, start=None, stop=None, **kwargs)
    
    def __str__(self):
        string = 'Circle({},{},{},'.format(self.x,self.y,self.rx)
        for key in self.kwargs.keys():
            if isinstance(self.kwargs[key],str):
                string += key + "='" + self.kwargs[key] + "',"
            else:
                string += key + '=' + self.kwargs[key] + ','
        if self.name != '':
            string += "name='" + self.name + "')"
        else:
            string = string[:-1] + ")"
        return string

In [10]:
r1=Rectangle(4,3,2,1,char='1',name='r1')
s1=Square(3,2,1,char='2')
l1=Line(4,3,2,1,char='3')
compound=CompoundShape([r1,s1,l1])
p1=Point(2,1,char='4')
t1=Triangle(6,5,4,3,2,1,char='5')
a1=Arc(6,5,4,3,0,math.pi,char='6')
o1=Oval(4,3,2,1,char='7')
c1=Circle(3,2,1,char='8')
print(str(r1))
print(str(s1))
print(str(l1))
print(str(compound))
print(str(p1))
print(str(t1))
print(str(a1))
print(str(o1))
print(str(c1))

Rectangle(4,3,2,1,char='1',name='r1')
Square(3,2,1,char='2')
Line(4,3,2,1,char='3')
CompoundShape([Rectangle(4,3,2,1,char='1',name='r1'),Square(3,2,1,char='2'),Line(4,3,2,1,char='3')])
Point(2,1,char='4')
Triangle(6,5,4,3,2,1,char='5')
Arc(6,5,4,3,0,3.141592653589793,char='6')
Oval(4,3,2,1,char='7')
Circle(3,2,1,char='8')


5. Add to `RasterDrawing` two functions, `save(filename)` and `load(filename)`. The save function writes the `__str__()` of all of the shapes in the drawing to a file (one shape per line). The load function, reads the file, and instantiates each object using the python `eval(expression)` function, and adds each shape to the drawing, thereby recreating a "saved" raster drawing. Use this functionality to save and load your happy face.

   `eval` takes a string that contains a fragment of a python code and executes it. Consider the following examples: 

In [11]:
eval("print('Hello')")

Hello


In [12]:
x = eval('1+2')
print(x)

3


In [13]:
class RasterDrawing:
    def __init__(self):
        self.shapes=dict()
        self.shape_names=list()
        
    def add_shape(self,shape):
        if shape.name == "":
            shape.name = self.assign_name()
        
        self.shapes[shape.name]=shape
        self.shape_names.append(shape.name)
        
    def paint(self,canvas):
        for shape_name in self.shape_names:
            self.shapes[shape_name].paint(canvas)
            
    def assign_name(self):
        name_base="shape"
        name = name_base+"_0"
        
        i=1
        while name in self.shapes:
            name = name_base+"_"+str(i)
            
        return name
    
    def save(self,filename):
        sf = open(filename, 'w')
        string = ""
        for shape_name in self.shape_names:
            string += str(self.shapes[shape_name])+'\n'
        sf.write(string[:-1])
        sf.close()
    
    def load(self,filename):
        sf = open(filename, 'r')
        for line in sf:
            self.add_shape(eval(line))
        sf.close()

In [14]:
rd=RasterDrawing()

rd.add_shape(Circle(15,15,15,char="1",name="head"))
rd.add_shape(Oval(10,18,2,4,char="2",name="left eye"))
rd.add_shape(Oval(20,18,2,4,char="3",name="right eye"))
rd.add_shape(Arc(15,15,10,10,math.pi,2*math.pi,char="4",name="mouth"))

rd.save("savedRasterDrawing")
del rd
rd = RasterDrawing()
rd.load("savedRasterDrawing")

c1=Canvas(31,31)

rd.paint(c1)

c1.display()

         111111111111          
       111          111        
      11              11       
    11                  11     
   11                    11    
   1      444444          1    
  1      44                1   
 11    44                  11  
 1     4      22222222      1  
11    4       2      2      11 
1    44       2      22      1 
1    4        22222222       1 
1    4            2          1 
1    4                       1 
1    4                       1 
1    4                       11
1    4                       1 
1    4                       1 
1    4        33333333       1 
1    44       3      3       1 
11    4       3      33     11 
 1     4      33333333      1  
 11    44         3        11  
  1      44                1   
   1      44444           1    
   11                    11    
    11                  11     
      11              11       
       111          111        
         111111111111          
               1               
